In [35]:
from glob import glob

for g in glob('./data/*.pdf'):
  print(g)

./data/2040_seoul_plan.pdf
./data/OneNYC_2050_Strategic_Plan.pdf


In [36]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


def read_pdf_and_split_text(pdf_path, chunk_size=1000, chunk_overlap=100):
  """
  주어진 PDF 파일을 읽고 텍스트를 분할합니다.

  매개변수:
    pdf_path (str): PDF 파일의 경로.
    chunk_size (int, 선택적): 각 텍스트 청크의 크기. 기본값은 1000
    chunk_overlap (int, 선택적): 청크 간의 중첩 크기. 기본값은 100
  반환값:
    list: 분할된 텍스트 청크의 리스트.
  """
  print(f"PDF {pdf_path} -----------------------------")

  pdf_loader = PyPDFLoader(pdf_path)
  data_from_pdf = pdf_loader.load()

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size, chunk_overlap=chunk_overlap
  )

  splits = text_splitter.split_documents(data_from_pdf)

  print(f"Number of splits: {len(splits)}\n")
  return splits

In [37]:
%pip install langchain_huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [38]:
%pip install torch==2.7.1 torchvision==0.22.1 torchaudio==2.7.1
%pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'mps'},
    encode_kwargs={'normalize_embeddings': True}
)

In [40]:
embeddings.embed_documents("안녕하세요")

[[0.002456949558109045,
  0.03226257860660553,
  -0.007424204610288143,
  0.005268458276987076,
  -0.058097369968891144,
  -0.030428607016801834,
  -0.00830006692558527,
  0.03674235939979553,
  0.009622698649764061,
  -0.007633970119059086,
  0.017204437404870987,
  0.03887702897191048,
  -0.02964901179075241,
  -0.02182001620531082,
  -0.0008699425961822271,
  -0.03256247937679291,
  0.03178023546934128,
  -0.022462841123342514,
  0.023502536118030548,
  -0.012333721853792667,
  -0.038810163736343384,
  -0.017907747998833656,
  0.03872930258512497,
  0.013715686276555061,
  0.025562601163983345,
  0.022995924577116966,
  -0.027591686695814133,
  0.027073394507169724,
  -0.00977582111954689,
  -0.026509791612625122,
  -0.006837903987616301,
  -0.02930246666073799,
  0.028823209926486015,
  -0.07535608857870102,
  -0.03274369239807129,
  -0.004341233056038618,
  -0.023116135969758034,
  0.021545208990573883,
  -0.05429801344871521,
  0.052812568843364716,
  0.043331343680620193,
  -0.0

In [42]:
from langchain_chroma import Chroma
import os

persist_directory = '../chroma_store'

if not os.path.exists(persist_directory):
  print("Creating new Chroma store")

  all_splits = []
  for g in glob('./data/*.pdf'):
    all_splits.extend(read_pdf_and_split_text(g))

  print(f"Total number of splits: {len(all_splits)}")

  vectorstore = Chroma.from_documents(
      documents=all_splits,
      embedding=embeddings,
      persist_directory=persist_directory
  )
else:
  print("Loading existing Chroma store")
  vectorstore = Chroma(
      persist_directory=persist_directory,
      embedding_function=embeddings
  )

Creating new Chroma store
PDF ./data/2040_seoul_plan.pdf -----------------------------
Number of splits: 308

PDF ./data/OneNYC_2050_Strategic_Plan.pdf -----------------------------


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Number of splits: 1023

Total number of splits: 1331



KeyboardInterrupt



KeyboardInterrupt: 

In [42]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

chunks = retriever.invoke("서울시 온실가스 저감 정책")

for chunk in chunks:
  print(chunk.metadata)
  print(chunk.page_content)